In [ ]:
from importlib import reload
#reload(Utilities)

import sys, os
import re
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from AbsPerfectPower import AbsPerfectPower as appwr 
#-----
from DOVSAudit import DOVSAudit
from DABatch import DABatch
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer

In [ ]:
# NOTE: Need to use DATE_DIFF with seconds because it will round down minutes
#       e.g., if delta = 5min29s, this will be collapsed to 5min, and will therefore pass
#             diff <= 5min

In [ ]:
def build_sql_v00(
    date_0        , 
    date_1        , 
    opco          , 
    td_sec        = 300, 
    group_by      = ['station_nb', 'circuit_nb'], 
    pd_ids        = ['3.26.0.47', '3.26.136.47', '3.26.136.66']
):
    r"""
    """
    #--------------------------------------------------
    accptbl_group_by = ['station_nb', 'station_nm', 'circuit_nb', 'circuit_nm', 'trsf_pole_nb']
    assert(set(group_by).difference(set(accptbl_group_by))==set())
    #--------------------------------------------------
    datetime_pattern = r"([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*"
    group_by_str = Utilities.join_list(
        list_to_join  = group_by, 
        quotes_needed = False, 
        join_str      = ', '
    )
    
    sql = """
    WITH MP AS (
        WITH COMP AS (
            SELECT opco_nb,opco_nm
            FROM default.company
        )
        SELECT
            MP.mfr_devc_ser_nbr,
            MP.prem_nb, \n{}, 
            COMP.opco_nm
    	FROM default.meter_premise MP
    	LEFT JOIN COMP ON MP.co_cd_ownr=COMP.opco_nb
    """.format(
        Utilities.join_list(
            list_to_join  = [f'        MP.{x}' for x in group_by], 
            quotes_needed = False, 
            join_str      = ', \n'
        )
    )
    if opco is not None:
        sql += "\tWHERE COMP.opco_nm = '{}'\n".format(opco)
    sql += "),"
    #-------------------------
    sql += """
    EDE_0 AS (
        SELECT
            EDE.serialnumber,
            CAST(regexp_replace(EDE.valuesinterval, '{}', '$1 $2') AS TIMESTAMP) AS valuesinterval,
            EDE.aep_premise_nb,
            MP.*
        FROM meter_events.end_device_event EDE
        INNER JOIN MP ON EDE.serialnumber=MP.mfr_devc_ser_nbr AND EDE.aep_premise_nb=MP.prem_nb
        WHERE CAST(regexp_replace(EDE.valuesinterval, '{}', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '{}' AND TIMESTAMP '{}'
        AND   EDE.enddeviceeventtypeid IN ({})
    """.format(
        datetime_pattern, 
        datetime_pattern, 
        date_0, 
        date_1, 
        Utilities.join_list(
            list_to_join  = pd_ids, 
            quotes_needed = True, 
            join_str      = ', '
        )
    )
    if opco is not None:
        sql += "    AND   EDE.aep_opco = '{}'\n".format(opco)
    sql += "),"
    #-------------------------
    sql += """
    EDE AS (
        SELECT *, 
            LAG(valuesinterval) OVER (PARTITION BY {} ORDER BY valuesinterval) AS valuesinterval_prev
        FROM
            EDE_0
    ),
    GroupedEntries AS (
        SELECT *,
        SUM(
            CASE 
                WHEN 
                    DATE_DIFF('second', valuesinterval_prev, valuesinterval) <= {} OR
                    valuesinterval_prev IS NULL
                THEN 0 
                ELSE 1 
            END
        ) 
        OVER (PARTITION BY {} ORDER BY valuesinterval) AS grp
        FROM EDE
    ),
    FinalCounts AS (
        SELECT\n{}, 
            grp,
            COUNT(serialnumber)            AS total_serialnumber,
            COUNT(DISTINCT serialnumber)   AS unique_serialnumber,
            COUNT(aep_premise_nb)          AS total_aep_premise_nb, 
            COUNT(DISTINCT aep_premise_nb) AS unique_aep_premise_nb, 
            MIN(valuesinterval)            AS min_valuesinterval, 
            MAX(valuesinterval)            AS max_valuesinterval
        FROM 
            GroupedEntries
        GROUP BY\n{}, 
            grp
    ), """.format(
        group_by_str, 
        td_sec, 
        group_by_str, 
        Utilities.join_list(
            list_to_join  = [f"        {x}" for x in group_by], 
            quotes_needed = False, 
            join_str      = ', \n'
        ), 
        Utilities.join_list(
            list_to_join  = [f"        {x}" for x in group_by], 
            quotes_needed = False, 
            join_str      = ', \n'
        )
    )
    #-------------------------
    sql += """
    TotalCounts AS (
        WITH COMP AS (
        	SELECT opco_nb,opco_nm
        	FROM default.company
        )
        SELECT\n{},
            COUNT(DISTINCT MP.mfr_devc_ser_nbr) AS group_SN_cnt,
            COUNT(DISTINCT MP.prem_nb)          AS group_PN_cnt
        FROM default.meter_premise MP
        LEFT JOIN COMP ON MP.co_cd_ownr=COMP.opco_nb
    """.format(
        Utilities.join_list(
            list_to_join  = [f'        MP.{x}' for x in group_by], 
            quotes_needed = False, 
            join_str      = ', \n'
        )
    )
    if opco is not None:
        sql += "\tWHERE COMP.opco_nm = '{}'".format(opco)
    sql += """
        GROUP BY
        	{}
    )""".format(
        Utilities.join_list(
            list_to_join  = [f'MP.{x}' for x in group_by], 
            quotes_needed = False, 
            join_str      = ', '
        )
    )
    #-------------------------
    sql += """
    SELECT 
        FinalCounts.*, 
        TotalCounts.group_SN_cnt, 
        TotalCounts.group_PN_cnt
    FROM 
        FinalCounts
    LEFT JOIN TotalCounts ON {}
    ORDER BY 
        {};
    """.format(
        Utilities.join_list(
            list_to_join  = [f"FinalCounts.{x}=TotalCounts.{x}" for x in group_by], 
            quotes_needed = False, 
            join_str      = ' AND '
        ), 
        Utilities.join_list(
            list_to_join  = [f"FinalCounts.{x}" for x in group_by], 
            quotes_needed = False, 
            join_str      = ', '
        )    
    )
    #--------------------------------------------------
    return sql 

In [ ]:
def build_sql_v0(
    date_0        , 
    date_1        , 
    opco          , 
    td_sec        = 300, 
    group_by      = ['station_nb', 'circuit_nb'], 
    pd_ids        = ['3.26.0.47', '3.26.136.47', '3.26.136.66']
):
    r"""
    """
    #--------------------------------------------------
    accptbl_group_by = ['station_nb', 'station_nm', 'circuit_nb', 'circuit_nm', 'trsf_pole_nb']
    assert(set(group_by).difference(set(accptbl_group_by))==set())
    #--------------------------------------------------
    datetime_pattern = r"([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*"
    group_by_str = Utilities.join_list(
        list_to_join  = group_by, 
        quotes_needed = False, 
        join_str      = ', '
    )
    
    sql = """
    WITH MP AS (
        WITH COMP AS (
            SELECT opco_nb,opco_nm
            FROM default.company
        )
        SELECT
            MP.mfr_devc_ser_nbr,
            MP.prem_nb, 
            CAST(COALESCE(NULLIF(TRIM(MP.latitude), ''),  NULLIF(TRIM(MP.latitude_nb), '')) AS DOUBLE)  AS latitude, 
            CAST(COALESCE(NULLIF(TRIM(MP.longitude), ''), NULLIF(TRIM(MP.longitude_nb), '')) AS DOUBLE) AS longitude, \n{}, 
            COMP.opco_nm
    	FROM default.meter_premise MP
    	LEFT JOIN COMP ON MP.co_cd_ownr=COMP.opco_nb
    """.format(
        Utilities.join_list(
            list_to_join  = [f'            MP.{x}' for x in group_by], 
            quotes_needed = False, 
            join_str      = ', \n'
        )
    )
    if opco is not None:
        sql += "\tWHERE COMP.opco_nm = '{}'\n".format(opco)
    sql += "),"
    #-------------------------
    sql += """
    EDE_0 AS (
        SELECT
            EDE.serialnumber,
            CAST(regexp_replace(EDE.valuesinterval, '{}', '$1 $2') AS TIMESTAMP) AS valuesinterval,
            EDE.aep_premise_nb,
            MP.*
        FROM meter_events.end_device_event EDE
        INNER JOIN MP ON EDE.serialnumber=MP.mfr_devc_ser_nbr AND EDE.aep_premise_nb=MP.prem_nb
        WHERE EDE.aep_event_dt BETWEEN '{}' AND '{}'
        AND   EDE.enddeviceeventtypeid IN ({})
    """.format(
        datetime_pattern, 
        date_0, 
        date_1, 
        Utilities.join_list(
            list_to_join  = pd_ids, 
            quotes_needed = True, 
            join_str      = ', '
        )
    )
    if opco is not None:
        sql += "    AND   EDE.aep_opco = '{}'\n".format(opco)
    sql += "),"
    #-------------------------
    sql += """
    EDE AS (
        SELECT *, 
            LAG(valuesinterval) OVER (PARTITION BY {} ORDER BY valuesinterval) AS valuesinterval_prev
        FROM
            EDE_0
    ),
    GroupedEntries AS (
        SELECT *,
        SUM(
            CASE 
                WHEN 
                    DATE_DIFF('second', valuesinterval_prev, valuesinterval) <= {} OR
                    valuesinterval_prev IS NULL
                THEN 0 
                ELSE 1 
            END
        ) 
        OVER (PARTITION BY {} ORDER BY valuesinterval) AS grp
        FROM EDE
    ),
    FinalCounts AS (
        SELECT\n{}, 
            grp,
            COUNT(serialnumber)            AS total_serialnumber,
            COUNT(DISTINCT serialnumber)   AS unique_serialnumber,
            COUNT(aep_premise_nb)          AS total_aep_premise_nb, 
            COUNT(DISTINCT aep_premise_nb) AS unique_aep_premise_nb, 
            MIN(valuesinterval)            AS min_valuesinterval, 
            MAX(valuesinterval)            AS max_valuesinterval, 
            AVG(latitude)                  AS avg_latitude, 
            AVG(longitude)                 AS avg_longitude
        FROM 
            GroupedEntries
        GROUP BY\n{}, 
            grp
    ), """.format(
        group_by_str, 
        td_sec, 
        group_by_str, 
        Utilities.join_list(
            list_to_join  = [f"        {x}" for x in group_by], 
            quotes_needed = False, 
            join_str      = ', \n'
        ), 
        Utilities.join_list(
            list_to_join  = [f"        {x}" for x in group_by], 
            quotes_needed = False, 
            join_str      = ', \n'
        )
    )
    #-------------------------
    sql += """
    TotalCounts AS (
        WITH COMP AS (
        	SELECT opco_nb,opco_nm
        	FROM default.company
        )
        SELECT\n{},
            COUNT(DISTINCT MP.mfr_devc_ser_nbr) AS group_SN_cnt,
            COUNT(DISTINCT MP.prem_nb)          AS group_PN_cnt
        FROM default.meter_premise MP
        LEFT JOIN COMP ON MP.co_cd_ownr=COMP.opco_nb
    """.format(
        Utilities.join_list(
            list_to_join  = [f'        MP.{x}' for x in group_by], 
            quotes_needed = False, 
            join_str      = ', \n'
        )
    )
    if opco is not None:
        sql += "\tWHERE COMP.opco_nm = '{}'".format(opco)
    sql += """
        GROUP BY
        	{}
    )""".format(
        Utilities.join_list(
            list_to_join  = [f'MP.{x}' for x in group_by], 
            quotes_needed = False, 
            join_str      = ', '
        )
    )
    #-------------------------
    sql += """
    SELECT 
        FinalCounts.*, 
        TotalCounts.group_SN_cnt, 
        TotalCounts.group_PN_cnt
    FROM 
        FinalCounts
    LEFT JOIN TotalCounts ON {}
    ORDER BY 
        {};
    """.format(
        Utilities.join_list(
            list_to_join  = [f"FinalCounts.{x}=TotalCounts.{x}" for x in group_by], 
            quotes_needed = False, 
            join_str      = ' AND '
        ), 
        Utilities.join_list(
            list_to_join  = [f"FinalCounts.{x}" for x in group_by], 
            quotes_needed = False, 
            join_str      = ', '
        )    
    )
    #--------------------------------------------------
    return sql 

In [ ]:
def build_sql(
    date_0        , 
    date_1        , 
    opco          , 
    group_by      = ['station_nb', 'circuit_nb'], 
    station_nbs   = None, 
    circuit_nbs   = None, 
    trsf_pole_nbs = None
):
    r"""
    """
    #--------------------------------------------------
    accptbl_group_by = ['station_nb', 'station_nm', 'circuit_nb', 'circuit_nm', 'trsf_pole_nb']
    assert(set(group_by).difference(set(accptbl_group_by))==set())
    #-------------------------
    build_sql_fncn = AMIEndEvents_SQL.build_sql_end_events
    #-------------------------
    build_sql_kwargs = dict(
        cols_of_interest      = TableInfos.AMIEndEvents_TI.std_columns_of_interest, 
        opco                  = opco, 
        datetime_range        = [date_0, date_1], 
        enddeviceeventtypeids = pdpu_ids, 
        join_mp_args          = dict(
            build_mp_kwargs = dict(
                cols_of_interest = TableInfos.MeterPremise_TI.std_columns_of_interest, 
                opco             = opco
            ), 
            join_type       = 'INNER'
        )
    )

In [ ]:
pd_ids   = ['3.26.0.47', '3.26.136.47', '3.26.136.66']
pu_ids   = ['3.26.0.216', '3.26.136.216']
pdpu_ids = pd_ids+pu_ids

In [ ]:
build_sql_fncn = AMIEndEvents_SQL.build_sql_end_events

In [ ]:
build_sql_kwargs = dict(
    cols_of_interest      = TableInfos.AMIEndEvents_TI.std_columns_of_interest, 
    opco                  = 'pso', 
    datetime_range        = ['2025-03-02', '2025-03-08'], 
    # datetime_range        = ['2025-04-01', '2025-04-02'], 
    enddeviceeventtypeids = pdpu_ids, 
    # enddeviceeventtypeids = None, 
    # join_mp_args          = True
    join_mp_args          = dict(
        build_mp_kwargs = dict(
            cols_of_interest = TableInfos.MeterPremise_TI.std_columns_of_interest, 
            opcos            = 'pso'
        ), 
        join_type       = 'LEFT'
    )
)

In [ ]:
ede = AMIEndEvents(
    df_construct_type         = DFConstructType.kRunSqlQuery,
    contstruct_df_args        = None, 
    init_df_in_constructor    = False, 
    build_sql_function        = build_sql_fncn, 
    build_sql_function_kwargs = build_sql_kwargs, 
    save_args                 = False
)

In [ ]:
# ede.df.to_pickle(r'C:\Users\s346557\Downloads\ede_df_2.pkl')

In [ ]:
print(ede.get_sql_statement())

In [ ]:
# ede.df['enddeviceeventtypeid'].isin(pdpu_ids).sum()

In [ ]:
# ede_df = ede.df
ede_df = pd.read_pickle(r'C:\Users\s346557\Downloads\ede_df_2.pkl')

In [ ]:
ede_df

In [ ]:
fuzziness = pd.Timedelta('5min')

In [ ]:
ede_df = Utilities_dt.strip_tz_info_and_convert_to_dt(
    df            = ede_df, 
    time_col      = 'valuesinterval', 
    placement_col = 'valuesinterval_local', 
    run_quick     = True, 
    n_strip       = 6, 
    inplace       = False
)

In [ ]:
ede_df = ede_df[ede_df['enddeviceeventtypeid'].isin(pd_ids)].sort_values(by=['valuesinterval']).copy()

In [ ]:
ede_df[['station_nb', 'circuit_nb', 'trsf_pole_nb']] = ede_df[['station_nb', 'circuit_nb', 'trsf_pole_nb']].replace('', np.nan)
ede_df = ede_df.dropna(subset=['station_nb', 'circuit_nb', 'trsf_pole_nb'])

In [ ]:
def helper_function(
    ede_df_i                 , 
    fuzziness                = pd.Timedelta('5min'), 
    SN_col                   = 'serialnumber', 
    PN_col                   = 'aep_premise_nb', 
    valuesinterval_local_col = 'valuesinterval_local'
):
    r"""
    """
    #--------------------------------------------------
    ede_df_i = Utilities_df.get_continuous_blocks_in_df_lite(
        df               = ede_df_i, 
        col_idfr         = valuesinterval_local_col, 
        data_sep         = fuzziness, 
        comp_operator    = '<=', 
        assert_unique    = False
    )
    #-------------------------
    return_df =  ede_df_i.groupby('block').agg({
        valuesinterval_local_col : ['mean', 'std', 'min', 'max'], 
        SN_col                   : [pd.Series.nunique, 'size'], 
        PN_col                   : [pd.Series.nunique, 'size'], 
    })
    #-------------------------
    return return_df

In [ ]:
meter_cnt = MeterPremise.get_meter_cnt(
    group_cols        = ['circuit_nb', 'station_nb'], 
    return_PN_cnt     = True, 
    opco              = 'pso', 
)

In [ ]:
meter_cnt

In [ ]:
ede_df['valuesinterval_local'].describe()

In [ ]:
start = time.time()
idk = ede_df.groupby(['station_nb', 'circuit_nb'])[ede_df.columns].apply(lambda x:
    helper_function(x)
)
print(time.time()-start)

In [ ]:
# start = time.time()
# idk = ede_df.groupby(['station_nb', 'circuit_nb', 'trsf_pole_nb'])[ede_df.columns].apply(lambda x:
#     helper_function(x)
# )
# print(time.time()-start)

In [ ]:
meter_cnt = meter_cnt.set_index(['station_nb', 'circuit_nb'])
idk2 = idk.copy().reset_index().set_index(['station_nb', 'circuit_nb'])

idk2 = Utilities_df.flatten_multiindex_columns(
    df            = idk2, 
    join_str      = ' ', 
    reverse_order = False, 
    to_ignore     = ['first'], 
    inplace       = False
)

idk2 = pd.merge(
    idk2, 
    meter_cnt, 
    how         = 'left', 
    left_index  = True, 
    right_index = True
)

# idk2['pct_SN'] = idk2['serialnumber nunique']/idk2['group_SN_cnt']
# idk2['pct_PN'] = idk2['aep_premise_nb nunique']/idk2['group_PN_cnt']

In [ ]:
idk2

In [ ]:
idk2 = idk2.rename(columns = {
    'valuesinterval_local min' : 'min_valuesinterval', 
    'valuesinterval_local max' : 'max_valuesinterval', 
    'serialnumber nunique'     : 'unique_serialnumber', 
    'serialnumber size'        : 'total_serialnumber', 
    'aep_premise_nb nunique'   : 'unique_aep_premise_nb', 
    'aep_premise_nb size'      : 'total_aep_premise_nb', 
}).reset_index().drop(columns=['block', 'valuesinterval_local mean', 'valuesinterval_local std'])
#-------------------------
idk2    = idk2.sort_values(by = ['station_nb', 'circuit_nb', 'min_valuesinterval', 'max_valuesinterval'], ignore_index=True)
idk2    = idk2.set_index(['station_nb', 'circuit_nb'])

In [ ]:
assert(0)

In [ ]:
conn_aws = Utilities.get_athena_prod_aws_connection()

In [ ]:
opco     = 'pso'
# group_by = ['station_nb', 'circuit_nb', 'trsf_pole_nb']
# group_by = ['station_nb', 'circuit_nb']
group_by = ['station_nm', 'circuit_nm']
# date_0   = '2025-03-02'
# date_1   = '2025-03-08'
date_0   = '2025-04-01'
date_1   = '2025-04-02'
# td_sec   = 300
td_sec   = 5

In [ ]:
# pd_ids   = ['3.26.0.47', '3.26.136.47', '3.26.136.66']
pd_ids   = ['3.26.0.47']

In [ ]:
sql = build_sql_v0(
    date_0        = date_0, 
    date_1        = date_1, 
    opco          = opco, 
    td_sec        = td_sec, 
    group_by      = group_by, 
    pd_ids        = pd_ids
)

In [ ]:
# print(sql)

In [ ]:
idk_sql = pd.read_sql_query(sql, conn_aws) 

In [ ]:
idk_sql_OG = idk_sql.copy()

In [ ]:
idk_sql[group_by] = idk_sql[group_by].replace('', np.nan)
idk_sql           = idk_sql.dropna(subset=group_by)
#-------------------------
idk_sql = idk_sql.drop(columns=['grp'])
idk_sql = idk_sql.sort_values(by = group_by+['min_valuesinterval', 'max_valuesinterval'], ignore_index=True)
idk_sql = idk_sql.set_index(group_by)
idk_sql = idk_sql.rename(columns = {'total_serialnumber' : 'total_events'}).drop(columns=['total_aep_premise_nb'])
idk_sql = Utilities_df.move_cols_to_front(
    df           = idk_sql, 
    cols_to_move = ['min_valuesinterval', 'max_valuesinterval']
)
#-------------------------
idk_sql['norm_total_events']          = idk_sql['total_events']/idk_sql['group_SN_cnt']
idk_sql['norm_unique_serialnumber']   = idk_sql['unique_serialnumber']/idk_sql['group_SN_cnt']
idk_sql['norm_unique_aep_premise_nb'] = idk_sql['unique_aep_premise_nb']/idk_sql['group_PN_cnt']

In [ ]:
idk_sql.shape

In [ ]:
idk_sql

In [ ]:
idk_sql[idk_sql['norm_unique_serialnumber'] > 0.5]

In [ ]:
idk_sql_gt25 = idk_sql[idk_sql['unique_serialnumber']>25].copy()

In [ ]:
# natsorted(idk_sql.index.get_level_values(0).unique())

In [ ]:
idk_sql_gt25

In [ ]:
idk_sql[idk_sql['group_SN_cnt']<10]

In [ ]:
idk_sql_gt25.loc['121ST & LYNN LANE']

In [ ]:
idk_sql_gt25.loc['CATOOSA']

In [ ]:
idk_station = idk_sql.groupby('station_nm').agg({
    'min_valuesinterval'    : 'min', 
    'max_valuesinterval'    : 'max', 
    'total_events'          : 'sum', 
    'unique_serialnumber'   : 'sum', 
    'unique_aep_premise_nb' : 'sum', 
    'group_SN_cnt'          : 'sum', 
    'group_PN_cnt'          : 'sum', 
})
#-------------------------
idk_station['norm_total_events']          = idk_station['total_events']/idk_station['group_SN_cnt']
idk_station['norm_unique_serialnumber']   = idk_station['unique_serialnumber']/idk_station['group_SN_cnt']
idk_station['norm_unique_aep_premise_nb'] = idk_station['unique_aep_premise_nb']/idk_station['group_PN_cnt']

In [ ]:
idk_station